### Info
Plotting comparisons

In [1]:
import numpy as np
import pandas as pd

### Load data into a DataFrame

In [2]:
min_ind = 1
max_ind = 2

model_names = ('hls','qkeras','qonnx_init','qonnx_mod')
pred_all = []   # will be a list of arrays from each model

for model_name in model_names:
    pred_single = np.array([])
    for file_index in range(min_ind, max_ind+1):
        data = np.load('./pred_GGHToGG_M125/preds_'+model_name+str(file_index)+'.npy')
        pred_single = np.append(pred_single,data)
        #print(pred_single.shape)
    pred_all.append(pred_single)

pred_all = np.array(pred_all).T

In [3]:
pred_all.shape

(4096, 4)

In [51]:
df = pd.DataFrame(pred_all, columns=model_names)
df['hls'].value_counts().sort_index()

hls
-128.000    2
-127.875    1
-127.750    5
-127.625    7
-127.500    1
           ..
 127.500    3
 127.625    3
 127.750    2
 127.875    1
 128.000    1
Name: count, Length: 1604, dtype: int64

### Compare predictions

In [5]:
df['qxinit_qkeras'] = df['qonnx_init'] - df['qkeras']
df['qxinit_qkeras'].value_counts()

qxinit_qkeras
0.0    4096
Name: count, dtype: int64

modified vs unmodified qonnx with respect to hls

In [6]:
df['qxinit_hls'] = df['qonnx_init'] - df['hls']
df['qxmod_hls'] = df['qonnx_mod'] - df['hls']

print(df['qxinit_hls'].value_counts())
print(df['qxmod_hls'].value_counts())

qxinit_hls
0.000       1909
256.000     1893
512.000      156
256.125       39
0.125         31
768.000       27
1024.000      19
1280.000      11
512.125        3
1536.000       3
768.125        2
1792.000       2
2048.000       1
Name: count, dtype: int64
qxmod_hls
0.000      1919
0.125        21
222.250       9
239.625       8
238.125       8
           ... 
156.625       1
137.250       1
132.750       1
36.125        1
183.625       1
Name: count, Length: 1014, dtype: int64


### Create histograms

In [7]:
import matplotlib.pyplot as plt

In [43]:
dfpl = pd.DataFrame([df['qxinit_hls'].value_counts(),df['qxmod_hls'].value_counts()])
dfpl = dfpl.transpose()
dfpl.columns = ['init','mod']
dfpl = dfpl.fillna(0)
dfpl /= dfpl['init'].sum()
dfpl *= 100
dfpl = dfpl.sort_index()
dfpl.index[500]

185.375

In [44]:
def categ(x):
    if x==0.0 : return '0.0'
    if x < 10. : return '0-10'
    if x < 150. : return '10-150'
    if x < 500. : return '150-500'
    return 'more'

In [45]:
# Categorize differences (deviations from hls preds)
dfpl['cat'] = dfpl.index.map(categ)

catdf = dfpl.groupby('cat').sum()
catdf

,init,mod
cat,,
0-10,0.756836,0.976562
0.0,46.606445,46.850586
10-150,0.000000,9.326172
150-500,47.167969,42.846680
more,5.468750,0.000000


In [42]:
catdf = catdf.reindex(['0.0','0-10','10-150','150-500','more'])
catdf

,init,mod
cat,,
0.0,46.606445,46.850586
0-10,0.756836,0.976562
10-150,0.000000,9.326172
150-500,47.167969,42.846680
more,5.468750,0.000000


In [ ]:
fig, ax = plt.subplots(layout='constrained')
width = 0.25
xticks = np.arange(catdf.shape[0])

for i in range(len(catdf.keys())):
    b = ax.bar(xticks+(-1+i)*width, catdf.values[:,i], width, label=catdf.keys()[i], align='edge')
    ax.bar_label(b, fmt='{:.2f}')

ax.legend()
ax.set_xticks(xticks)
ax.set_xticklabels(catdf.index.to_list())
ax.set_yscale('log')
ax.xaxis.set_tick_params(length=0)